In [1]:
from datasets import load_from_disk
text_dataset2 = load_from_disk("finer-139-revised/tapt_text_dataset")
text_dataset2


Dataset({
    features: ['text'],
    num_rows: 900384
})

In [9]:
from transformers import BertTokenizerFast
from datasets import load_from_disk

# Lade Tokenizer und Dataset
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
text_dataset = load_from_disk("finer-139-revised/tapt_text_dataset")

token_counts = [
    (i, len(tokenizer.encode(row["text"], add_special_tokens=False)))
    for i, row in enumerate(text_dataset2)
]


Token indices sequence length is longer than the specified maximum sequence length for this model (628 > 512). Running this sequence through the model will result in indexing errors


In [22]:
# Sort by token count
sorted_by_tokens = sorted(token_counts, key=lambda x: x[1])
top_10_tokens = sorted_by_tokens[9000:10000]  # top 10 shortest

# Output
for idx, token_count in top_10_tokens:
    #print(f"BERT-Tokens (inkl. CLS/SEP): {token_count}")
    print(text_dataset2[idx]['text'])
    #print("="*80)

See Note 12 for further information .
See Note 14 ( b ) .
See Note 12 for further details .
44 Table of Contents Item 2 .
21 Table of Contents Item 2 .
30 Table of Contents Item 2 .
36 Table of Contents ITEM 2 .
16 Table of Contents Item 2 .
§ 9606 ( a ) .
25 Table of Contents ITEM 2 .
38 Table of Contents Item 2 .
47 Table of Contents Item 3 .
23 Table of Contents Item 2 .
See further discussion in Note 14 .
84 Table of Contents Item 2 .
See Note 17 entitled Discontinued Operations .
See Note 20 for further information .
See Note 11 for further information .
See Note 13 for additional information .
See Note 13 for further information .
27 Table of Contents Item 2 .
41 Table of Contents Item 2 .
35 Table of Contents Item 2 .
25 TABLE OF CONTENTS ITEM 2 .
31 Table of Contents ITEM 2 .
35 Table of Contents Item 2 .
15 Table of Contents Item 2 .
See Note 12 for additional information .
78 Table of Contents ITEM 2 .
32 Table of Contents ITEM 2 .
( 4 ) See note 8 .
( 4 ) See note 8 .
No opt

# Count

In [25]:
# 1) Tokenlängen EINMAL berechnen (parallel & batched)
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def add_len(batch):
    out = tokenizer(
        batch["text"],
        add_special_tokens=True,
        truncation=False,            # kein Abschneiden, keine 512-Warnungen
        padding=False,
        return_attention_mask=False,
        return_token_type_ids=False,
    )
    return {"token_len": [len(x) for x in out["input_ids"]]}

text_dataset2 = text_dataset2.map(add_len, batched=True, num_proc=4, desc="Tokenlängen")
# Optional: dauerhaft speichern, damit spätere Sessions nichts neu rechnen
# text_dataset2.save_to_disk("text_dataset2_with_lens")


Tokenlängen (num_proc=4):   0%|          | 0/900384 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (632 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (576 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (630 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (741 > 512). Running this sequence through the model will result in indexing errors


In [34]:
def mask_less_than_10(batch):
    l = batch["token_len"]
    return [(x < 10) for x in l]
    
def mask_10_48(batch):
    l = batch["token_len"]
    return [(10 < x < 48) for x in l]
    
def mask_48_64(batch):
    l = batch["token_len"]
    return [(48 < x < 64) for x in l]

def mask_64_128(batch):
    l = batch["token_len"]
    return [(64 < x < 128) for x in l]

def mask_128_192(batch):
    l = batch["token_len"]
    return [(128 < x < 192) for x in l]


def mask_192_256(batch):
    l = batch["token_len"]
    return [(192 < x < 256) for x in l]

def mask_256_512(batch):
    l = batch["token_len"]
    return [(256 < x < 512) for x in l]

def mask_ge_512(batch):
    l = batch["token_len"]
    return [x >= 512 for x in l]


count_less_than_10 = text_dataset2.filter(mask_less_than_10, batched=True, num_proc=4).num_rows
count_10_48 = text_dataset2.filter(mask_10_48, batched=True, num_proc=4).num_rows
count_48_64 = text_dataset2.filter(mask_48_64, batched=True, num_proc=4).num_rows
count_64_128 = text_dataset2.filter(mask_64_128, batched=True, num_proc=4).num_rows
count_128_192 = text_dataset2.filter(mask_128_192, batched=True, num_proc=4).num_rows
count_192_256 = text_dataset2.filter(mask_192_256, batched=True, num_proc=4).num_rows
count_256_512 = text_dataset2.filter(mask_256_512, batched=True, num_proc=4).num_rows
count_bigger_than_512  = text_dataset2.filter(mask_ge_512, batched=True, num_proc=4).num_rows

In [37]:
# Print overview of counted Token Length categories
print("===== Counted Token Length Count Overview=====")
print(f"> 512         : {count_bigger_than_512}")
print(f"256 - 512     : {count_256_512}")
print(f"192 - 256     : {count_192_256}")
print(f"128 - 192     : {count_128_192}")
print(f"64  - 128     : {count_64_128}")
print(f"48  - 64      : {count_48_64}")
print(f"10  - 48      : {count_10_48}")
print(f"< 10          : {count_less_than_10}")
print("================================")
print(f"10  - 192      : {count_10_48+count_48_64+count_64_128+count_128_192}")

===== Counted Token Length Count Overview=====
> 512         : 379
256 - 512     : 2985
192 - 256     : 5398
128 - 192     : 24995
64  - 128     : 181523
48  - 64      : 146765
10  - 48      : 505454
< 10          : 9889
10  - 192      : 858737
